# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,adamstown,-25.0660,-130.1015,24.46,78,99,3.22,PN,1704332821
1,1,saint-memmie,48.9525,4.3841,10.01,78,97,9.26,FR,1704332821
2,2,blackmans bay,-43.0167,147.3167,18.18,69,35,0.45,AU,1704332822
3,3,port-aux-francais,-49.3500,70.2167,4.47,68,69,14.49,TF,1704332822
4,4,pariaman,-0.6190,100.1200,28.03,82,98,0.87,ID,1704332822


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
city_map = city_data_df.hvplot.points("Lng", 
    "Lat", 
    geo = True, 
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha = 0.5,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500
                                     )

# Display the map
city_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [8]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_cities_df = city_data_df.loc[(city_data_df["Max Temp"] < 27) & (city_data_df["Max Temp"] > 21) 
                  & (city_data_df["Cloudiness"] == 0) & (city_data_df["Wind Speed"] < 4.5)
                                      ]

# Drop any rows with null values
ideal_cities_df.dropna()

# Display sample data
ideal_cities_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
67,67,saint-pierre,-21.3393,55.4781,25.82,78,0,0.51,RE,1704332841
105,105,margaret river,-33.9500,115.0667,22.10,46,0,3.34,AU,1704332851
233,233,villa maria grande,-31.6656,-59.9018,23.20,73,0,4.31,AR,1704332882
260,260,ixtapa,20.7000,-105.2000,24.57,65,0,2.57,MX,1704332888
285,285,jeddah,21.5169,39.2192,21.92,60,0,1.54,SA,1704332894


### Step 3: Create a new DataFrame called `hotel_df`.

In [9]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_cities_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
67,saint-pierre,RE,-21.3393,55.4781,78,
105,margaret river,AU,-33.9500,115.0667,46,
233,villa maria grande,AR,-31.6656,-59.9018,73,
260,ixtapa,MX,20.7000,-105.2000,65,
285,jeddah,SA,21.5169,39.2192,60,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [10]:
# Set parameters to search for a hotel
radius = 10000
params = {"categories": "accommodation.hotel",
          "apiKey":geoapify_key
         }


# Print a message to follow up the hotel search
print("Starting hotel search")
print('----------------------------')

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lon = row["Lng"]
    lat = row["Lat"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] =  f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params = params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
----------------------------
saint-pierre - nearest hotel: Tropic Hotel
margaret river - nearest hotel: Margaret River Hotel
villa maria grande - nearest hotel: No hotel found
ixtapa - nearest hotel: Ma. Cristina
jeddah - nearest hotel: No hotel found
cabo san lucas - nearest hotel: Comfort Rooms
les trois-ilets - nearest hotel: Suite Villa
ovalle - nearest hotel: Hotel Roxy
san justo - nearest hotel: Hotel España
sinfra - nearest hotel: Complexe wilson


,City,Country,Lat,Lng,Humidity,Hotel Name
67,saint-pierre,RE,-21.3393,55.4781,78,Tropic Hotel
105,margaret river,AU,-33.9500,115.0667,46,Margaret River Hotel
233,villa maria grande,AR,-31.6656,-59.9018,73,No hotel found
260,ixtapa,MX,20.7000,-105.2000,65,Ma. Cristina
285,jeddah,SA,21.5169,39.2192,60,No hotel found
335,cabo san lucas,MX,22.8909,-109.9124,61,Comfort Rooms
412,les trois-ilets,MQ,14.5390,-61.0361,83,Suite Villa
501,ovalle,CL,-30.5983,-71.2003,63,Hotel Roxy
516,san justo,AR,-30.7891,-60.5919,74,Hotel España
529,sinfra,CI,6.6167,-5.9167,91,Complexe wilson


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [11]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points("Lng", 
    "Lat", 
    geo = True, 
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha = 0.5,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    hover_cols = ["Hotel Name", "Country"]
                                  )

# Display the map
hotel_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)